In [19]:
import sys
sys.path.append("../scripts")

import data
import utils
import models
import experiments

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import optuna

In [73]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = data.get_fold("german", 0)

In [74]:
args = {
    "dataset" : "german",
    "alpha" : 0.1,
    "n_groups" : 4,
    "model_name" : "M2FGB_eod",
    "n_trials" : 100,
    "thresh" : "ks"
}

In [75]:
col_trans = ColumnTransformer(
    [
        ("numeric", StandardScaler(), data.NUM_FEATURES[args["dataset"]]),
        (
            "categorical",
            OneHotEncoder(
                drop="if_binary", sparse_output=False, handle_unknown="ignore"
            ),
            data.CAT_FEATURES[args["dataset"]],
        ),
    ],
    verbose_feature_names_out=False,
)
col_trans.set_output(transform="pandas")

ColumnTransformer(transformers=[('numeric', StandardScaler(),
                                 ['Age', 'CreditAmount', 'Dependents',
                                  'Duration', 'ExistingCredits',
                                  'InstallmentRate', 'ResidenceSince']),
                                ('categorical',
                                 OneHotEncoder(drop='if_binary',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['CheckingAccount', 'CreditHistory', 'Purpose',
                                  'SavingsAccount', 'EmploymentSince', 'Gender',
                                  'OtherDebtors', 'Property',
                                  'OtherInstallmentPlans', 'Housing', 'Job',
                                  'Telephone', 'ForeignWorker'])],
                  verbose_feature_names_out=False)

In [76]:
A_train, A_val, A_test = experiments.get_subgroup_feature(
    args["dataset"], X_train, X_val, X_test, args["n_groups"]
)

In [77]:
preprocess = Pipeline([("preprocess", col_trans)])
preprocess.fit(X_train)
X_train = preprocess.transform(X_train)
X_val = preprocess.transform(X_val)
X_test = preprocess.transform(X_test)

In [78]:
model_class = experiments.get_model(args["model_name"])

In [79]:
scorer = utils.get_combined_metrics_scorer(
    alpha=args["alpha"], performance_metric="bal_acc", fairness_metric="min_eod"
)

In [80]:
study = optuna.create_study(direction="maximize")
objective = lambda trial: experiments.run_trial(
    trial,
    scorer,
    X_train,
    Y_train,
    A_train,
    X_val,
    Y_val,
    A_val,
    model_class,
    experiments.get_param_spaces(args["model_name"]),
    args,
)
study.optimize(objective, n_trials=args["n_trials"])
best_params = study.best_params.copy()

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Using self-defined objective function
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [81]:
model = model_class(**study.best_params)
model.fit(X_train, Y_train, A_train)
y_val_score = model.predict_proba(X_val)[:, 1]
thresh = utils.get_best_threshold(Y_val, y_val_score)
y_test_score = model.predict_proba(X_test)[:, 1]
y_test_pred = y_test_score > thresh

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Using self-defined objective function
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [82]:
eod = utils.equal_opportunity_score(Y_test, y_test_pred, A_test)
min_eod = utils.min_equal_opportunity_score(Y_test, y_test_pred, A_test)

eod, 1 - min_eod

(0.25, 0.75)

In [93]:
scorer = utils.get_combined_metrics_scorer(
    alpha=args["alpha"], performance_metric="bal_acc", fairness_metric="min_eod"
)
study = optuna.create_study(direction="maximize")
objective = lambda trial: experiments.run_trial(
    trial,
    scorer,
    X_train,
    Y_train,
    A_train,
    X_val,
    Y_val,
    A_val,
    model_class,
    experiments.get_param_spaces(args["model_name"]),
    args,
)
study.optimize(objective, n_trials=args["n_trials"])
best_params = study.best_params.copy()

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Using self-defined objective function
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [92]:
best_params

{'min_child_weight': 664.817571944248,
 'n_estimators': 304,
 'learning_rate': 0.1985351394744189,
 'max_depth': 6,
 'reg_lambda': 0.03322712909363115,
 'fair_weight': 2.244058626509359}

In [84]:
model = model_class(**study.best_params)
model.fit(X_train, Y_train, A_train)
y_val_score = model.predict_proba(X_val)[:, 1]
thresh = utils.get_best_threshold(Y_val, y_val_score)
y_test_score = model.predict_proba(X_test)[:, 1]
y_test_pred = y_test_score > thresh

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Using self-defined objective function
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] St

In [85]:
eod = utils.equal_opportunity_score(Y_test, y_test_pred, A_test)
min_eod = utils.min_equal_opportunity_score(Y_test, y_test_pred, A_test)

eod, 1 - min_eod

(0.0, 0.0)

In [86]:
for a in np.unique(A_test):
    print(y_test_pred[(A_test == a) & (Y_test == 1)].mean())

0.0
0.0
0.0
0.0
